In [1]:
import shutil

from core.gain import *
from core.rnn_predic import *
from core.models import *
from core.util import *
#from core.window import WindowGenerator, MissData, make_dataset_water, WaterDataGenerator
from core.window import WindowGenerator, make_dataset_gain, make_dataset_water
from core.file_open import make_dataframe
from core.miss_data import MissData
import json
%matplotlib widget
import os

In [2]:
def hour_to_day_mean(array):
    time = 24
    array = array.reshape((array.shape[0], array.shape[1] // time, time, array.shape[2]))
    array = array.mean(2)
    return array

In [3]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    width = input_width + label_width
    length = df.shape[0] - width
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    

    predictions = model(inputs).numpy()
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    plt.figure()
    #input_index = np.array(range(0, length, 24))
    input_index = np.array(range(inputs_day.shape[0]))
    #label_index = input_index + 24* (7 + predict_day)
    label_index = input_index + (7 + predict_day)
    plt.plot(input_index, inputs_day[:, 0, :], label='input')
    plt.plot(label_index, label_day[:, predict_day, :], label='label')
    plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    plt.legend()
    #plt.savefig('test_plt.png')
    
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    o_ = np.mean(label_day[:,predict_day,:])
    p_ = np.mean(pred_day[:,predict_day,:])
    
    oi = label_day[:,predict_day,:]
    pi = pred_day[:,predict_day,:]
    
    high = ((oi-o_)*(pi-p_)).sum()
    low = np.sqrt( ( (oi-o_)**2 ).sum() )
    low = low * np.sqrt( ( (pi-p_)**2 ).sum() )
    
    R = high/low
    RS = (R)**2
    
    return nse3, pbias3, pred_day, label_day, mae, rmse, RS, R

In [4]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e) 


1 Physical GPUs, 1 Logical GPUs


In [5]:
data_path = 'data/'
parameters_dir = 'input'

parameters_file = 'input.json'
parameters_path = '{dir}/{file}'.format(dir=parameters_dir, file=parameters_file)

with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

gain_parameters = parameters['gain']
rnn_parameters = parameters['rnn']
file_parameters = parameters['file']

In [6]:
file_parameters['watershed']

'yeong_pull'

In [7]:
parameters_path = parameters_dir+'/'+ file_parameters['watershed'] + '.json'
with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

In [8]:
pd.set_option('display.max_columns', 1000)

In [9]:
data_parameters = parameters['data']

interpolation_option = data_parameters['interpolation']
colum_idx = data_parameters['columns']
watershed = data_parameters['watershed']
file_names = data_parameters['files']
folder = data_parameters['directorys']
for i in range(len(folder)):
    folder[i] = watershed+folder[i]

In [10]:
__GAIN_TRAINING__ = gain_parameters['train']
gain_epochs = gain_parameters['max_epochs']
gain_in_setps = gain_parameters['input_width']
gain_out_setps = gain_parameters['label_width']
gain_batch_size = gain_parameters['batch_size']
gain_fill_no = gain_parameters['fill_width']
gain_shift = gain_parameters['shift_width']
gain_miss_rate = gain_parameters['miss_rate']

__RNN_TRAINING__ = rnn_parameters['train']
rnn_epochs = rnn_parameters['max_epochs']
rnn_in_setps = rnn_parameters['input_width']
rnn_out_steps = rnn_parameters['label_width']
rnn_batch_size = rnn_parameters['batch_size']
rnn_predict_day = rnn_parameters['predict_day']
rnn_target_column = rnn_parameters['target_column']

if rnn_predict_day < 3 or rnn_predict_day >5:
    print('predict_day err')
    exit(88)
rnn_predict_day -= 1

In [11]:
__GAIN_TRAINING__, gain_epochs ,__RNN_TRAINING__, rnn_epochs

(True, 2000, True, 15)

In [12]:
run_num = range(len(folder))
#if file_parameters['watershed'] == 'nak':
#    run_num = range(len(folder))
#else:
#    run_num = [0]

real_df_all = pd.DataFrame([])
target_all = target_mean = target_std = 0

gain_val_performance = {}
gain_performance = {}

length = len(run_num)

ddday = 31
mmmonth = 12

#length = 3

In [13]:
for i in range(length):

    idx = run_num[i]

    print('interpol flag : ', interpolation_option[idx])
    print('folder : ', data_path + folder[idx])
    print('colum_idx : ', colum_idx[idx])
    print('file_names[idx] : ', file_names[idx])

    #df, times = make_dataframe(data_path+folder[idx], file_names[idx], colum_idx[idx], interpolation=interpolation_option[idx])
    df, times, mmmonth, ddday = make_dataframe(data_path+folder[idx], file_names[idx], 
                               colum_idx[idx], interpolation=interpolation_option[idx],
                              first_file_no=i, month=mmmonth, day=ddday)

    df_all, train_mean, train_std, df = normalize(df)


    if i == 0:
        dfff = df
        target_all = df_all
        target_std = train_std
        target_mean = train_mean
        start_year = str(times.iloc[0].year)
        end_year = str(times.iloc[-1].year)

    if interpolation_option[idx][0] == False:

        loadfiles = ['idx.npy', 'miss.npy', 'discriminator.h5', 'generator.h5']

        gain_calc_falg = True

        if __GAIN_TRAINING__ == True:
            gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
            #print(folder[idx], ': training ', 'Miss date save : ', gain_calc_falg)
        else:
            for file in loadfiles:
                if os.path.isfile('save/' + folder[idx]+file):
                    shutil.copyfile('save/' + folder[idx]+file, 'save/'+file)
                    #print('load file name : save/' + folder[idx]+file)
                else:
                    if file == 'miss.npy':
                        gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
                        #print(folder[idx], ': is not miss.npy ', 'Miss date save : ', gain_calc_falg)

        if gain_calc_falg == True:
            #print('GainWindowGenerator in main')
            WindowGenerator.make_dataset = make_dataset_gain
            wide_window = WindowGenerator(input_width=gain_in_setps, label_width=gain_out_setps, shift=gain_shift,
                                          fill_no=gain_fill_no, miss_rate=gain_miss_rate, batch_size=gain_batch_size,
                                          train_df = df_all, val_df = df_all, test_df = df_all, df = df)

            #gain = model_GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False, epochs=gain_epochs, training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')
            gain = model_GAIN(shape=(gain_in_setps, df_all.shape[1]), gen_sigmoid=False, epochs=gain_epochs,
                              training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')

            gain_val_performance[str(i)] = gain.evaluate(wide_window.val)
            gain_performance[str(i)] = gain.evaluate(wide_window.test, verbose=0)

            #print('file proc in main')
            if __GAIN_TRAINING__ == True:
                #dir = 'save/'+folder[i]
                if not os.path.exists('save/' + folder[idx]):
                    os.makedirs('save/'+folder[idx])
                for file in loadfiles:
                    shutil.copyfile('save/' + file, 'save/' + folder[idx] + file)

            #print('create_dataset_with_gain in main')
            #ori, gan = create_dataset_with_gain(gain=gain, window=wide_window, df=df)
            ori, gan = create_dataset_with_gain(gain=gain, shape=(gain_in_setps, df_all.shape[1]), df=df)

        else:
            gan = create_dataset_interpol(window=gain_in_setps, df=df)
    else:
        gan = create_dataset_interpol(window=gain_in_setps, df=df)

    if i == 0 :
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.DataFrame(gan)
    else:
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.concat([real_df_all, pd.DataFrame(gan)], axis=1)

interpol flag :  [False, False]
folder :  data/yeong/자동/
colum_idx :  :,[26,27,28,29,30,31,32,33]
file_names[idx] :  [['나주_2016.xlsx', '나주_2017.xlsx', '나주_2018.xlsx'], ['지석천_2016.xlsx', '지석천_2017.xlsx', '지석천_2018.xlsx'], ['용봉_2016.xlsx', '용봉_2017.xlsx', '용봉_2018.xlsx'], ['우치_2016.xlsx', '우치_2017.xlsx', '우치_2018.xlsx']]
Epoch 1/2000
1/1 [==============================] - 0s 256ms/step - gen_loss: 116.5952 - disc_loss: 0.7191 - rmse: 1.0155 - val_loss: 0.9373
Epoch 2/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 94.1617 - disc_loss: 0.5254 - rmse: 0.9486 - val_loss: 0.8830
Epoch 3/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 84.7016 - disc_loss: 0.5055 - rmse: 0.8909 - val_loss: 0.8822
Epoch 4/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 80.8677 - disc_loss: 0.4063 - rmse: 0.8987 - val_loss: 0.8798
Epoch 5/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 64.0129 - disc_loss: 0.3845 - rmse: 0.7715 -

1/1 [==============================] - 0s 21ms/step - gen_loss: 10.7814 - disc_loss: 0.3401 - rmse: 0.3882 - val_loss: 0.3176
Epoch 58/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.8081 - disc_loss: 0.3284 - rmse: 0.4026 - val_loss: 0.3633
Epoch 59/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.7221 - disc_loss: 0.3271 - rmse: 0.3762 - val_loss: 0.4013
Epoch 60/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.8024 - disc_loss: 0.3428 - rmse: 0.4156 - val_loss: 0.3347
Epoch 61/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.4000 - disc_loss: 0.3382 - rmse: 0.3984 - val_loss: 0.3356
Epoch 62/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 10.5552 - disc_loss: 0.3243 - rmse: 0.4179 - val_loss: 0.3435
Epoch 63/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.7511 - disc_loss: 0.3317 - rmse: 0.4073 - val_loss: 0.3393
Epoch 64/2000
1/1 [====================

Epoch 116/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 7.5295 - disc_loss: 0.2533 - rmse: 0.3380 - val_loss: 0.3173
Epoch 117/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.3232 - disc_loss: 0.2685 - rmse: 0.3119 - val_loss: 0.3213
Epoch 118/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.5718 - disc_loss: 0.2652 - rmse: 0.2870 - val_loss: 0.3368
Epoch 119/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.7809 - disc_loss: 0.2655 - rmse: 0.3049 - val_loss: 0.3524
Epoch 120/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.8968 - disc_loss: 0.2587 - rmse: 0.3768 - val_loss: 0.3557
Epoch 121/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.6999 - disc_loss: 0.2663 - rmse: 0.3809 - val_loss: 0.2855
Epoch 122/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.3091 - disc_loss: 0.2672 - rmse: 0.3710 - val_loss: 0.3242
Epoch 123/2000
1/1 [

1/1 [==============================] - 0s 20ms/step - gen_loss: 5.2713 - disc_loss: 0.2535 - rmse: 0.2862 - val_loss: 0.3571
Epoch 175/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.3643 - disc_loss: 0.2457 - rmse: 0.3077 - val_loss: 0.3177
Epoch 176/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.0805 - disc_loss: 0.2391 - rmse: 0.2770 - val_loss: 0.2885
Epoch 177/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.3441 - disc_loss: 0.2183 - rmse: 0.2907 - val_loss: 0.3423
Epoch 178/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.2089 - disc_loss: 0.2370 - rmse: 0.2865 - val_loss: 0.3090
Epoch 179/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.3962 - disc_loss: 0.2189 - rmse: 0.3076 - val_loss: 0.3192
Epoch 180/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.6260 - disc_loss: 0.2426 - rmse: 0.3204 - val_loss: 0.3465
Epoch 181/2000
1/1 [===============

1/1 [==============================] - 0s 21ms/step - gen_loss: 5.2324 - disc_loss: 0.2246 - rmse: 0.3008 - val_loss: 0.2939
Epoch 233/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.4446 - disc_loss: 0.2284 - rmse: 0.3188 - val_loss: 0.2898
Epoch 234/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.9892 - disc_loss: 0.2313 - rmse: 0.2800 - val_loss: 0.2835
Epoch 235/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.7943 - disc_loss: 0.2233 - rmse: 0.3247 - val_loss: 0.3127
Epoch 236/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.5948 - disc_loss: 0.2182 - rmse: 0.2954 - val_loss: 0.3386
Epoch 237/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.4133 - disc_loss: 0.2163 - rmse: 0.2999 - val_loss: 0.3432
Epoch 238/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.2453 - disc_loss: 0.2270 - rmse: 0.2790 - val_loss: 0.3275
Epoch 239/2000
1/1 [===============

...

1/1 [==============================] - 0s 19ms/step - gen_loss: 0.7677 - disc_loss: 0.0906 - rmse: 0.0753 - val_loss: 0.0686
Epoch 1802/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.6480 - disc_loss: 0.0878 - rmse: 0.0850 - val_loss: 0.1116
Epoch 1803/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.7408 - disc_loss: 0.0931 - rmse: 0.0913 - val_loss: 0.1023
Epoch 1804/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.7935 - disc_loss: 0.0891 - rmse: 0.1087 - val_loss: 0.0979
Epoch 1805/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.8084 - disc_loss: 0.0849 - rmse: 0.0879 - val_loss: 0.0648
Epoch 1806/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.7542 - disc_loss: 0.0894 - rmse: 0.0965 - val_loss: 0.1065
Epoch 1807/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.9071 - disc_loss: 0.0875 - rmse: 0.1634 - val_loss: 0.0967
Epoch 1808/2000
1/1 [========

1/1 [==============================] - 0s 19ms/step - gen_loss: 0.9490 - disc_loss: 0.0963 - rmse: 0.1087 - val_loss: 0.0765
Epoch 1859/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.8788 - disc_loss: 0.0859 - rmse: 0.1103 - val_loss: 0.1808
Epoch 1860/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.9620 - disc_loss: 0.0887 - rmse: 0.1329 - val_loss: 0.2113
Epoch 1861/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 1.0154 - disc_loss: 0.0960 - rmse: 0.1221 - val_loss: 0.1052
Epoch 1862/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.9415 - disc_loss: 0.0882 - rmse: 0.0841 - val_loss: 0.1391
Epoch 1863/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.8702 - disc_loss: 0.0900 - rmse: 0.1160 - val_loss: 0.1121
Epoch 1864/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 1.0006 - disc_loss: 0.0985 - rmse: 0.1290 - val_loss: 0.1013
Epoch 1865/2000
1/1 [========

1/1 [==============================] - 0s 19ms/step - gen_loss: 0.7327 - disc_loss: 0.0943 - rmse: 0.0923 - val_loss: 0.1025
Epoch 1917/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.8352 - disc_loss: 0.0898 - rmse: 0.1195 - val_loss: 0.1080
Epoch 1918/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.8663 - disc_loss: 0.0865 - rmse: 0.0990 - val_loss: 0.1001
Epoch 1919/2000
1/1 [==============================] - 0s 25ms/step - gen_loss: 0.7525 - disc_loss: 0.0860 - rmse: 0.1090 - val_loss: 0.1160
Epoch 1920/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.8955 - disc_loss: 0.0933 - rmse: 0.1541 - val_loss: 0.0990
Epoch 1921/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.8338 - disc_loss: 0.0831 - rmse: 0.0945 - val_loss: 0.1477
Epoch 1922/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.9397 - disc_loss: 0.0927 - rmse: 0.1552 - val_loss: 0.0894
Epoch 1923/2000
1/1 [========

1/1 [==============================] - 0s 19ms/step - gen_loss: 0.7445 - disc_loss: 0.0905 - rmse: 0.1060 - val_loss: 0.1361
Epoch 1975/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.9242 - disc_loss: 0.0927 - rmse: 0.1238 - val_loss: 0.1090
Epoch 1976/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.7222 - disc_loss: 0.1010 - rmse: 0.0978 - val_loss: 0.1024
Epoch 1977/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 0.7287 - disc_loss: 0.0905 - rmse: 0.0882 - val_loss: 0.1065
Epoch 1978/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 0.6499 - disc_loss: 0.0890 - rmse: 0.1180 - val_loss: 0.1095
Epoch 1979/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.7522 - disc_loss: 0.0934 - rmse: 0.1243 - val_loss: 0.0817
Epoch 1980/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.6019 - disc_loss: 0.0916 - rmse: 0.0720 - val_loss: 0.1065
Epoch 1981/2000
1/1 [========

row in
row in
row in
row in
row in
row in


In [14]:
#-----------------------------------------------------------------------------------------------------------------
#            RRRRRRRRRRRRRRRRRR          NNNNNN       NNNNNN      NNNNNN       NNNNNN
#           RRRRRRRRRRRRRRRRRRRR        NNNNNNN      NNNNNN      NNNNNNN      NNNNNN
#          RRRRRRR      RRRRRRRR       NNNNNNNN     NNNNNN      NNNNNNNN     NNNNNN
#         RRRRRRR      RRRRRRRR       NNNNNNNNN    NNNNNN      NNNNNNNNN    NNNNNN
#        RRRRRRRRRRRRRRRRRRRR        NNNNNNNNNN   NNNNNN      NNNNNNNNNN   NNNNNN
#       RRRRRRRRRRRRRRRRR           NNNNNNNNNNN  NNNNNN      NNNNNNNNNNN  NNNNNN 
#      RRRRRRR     RRRRRRR         NNNNNNN NNNN NNNNNN      NNNNNNN NNNN NNNNNN
#     RRRRRRR       RRRRRRR       NNNNNNN  NNNNNNNNNN      NNNNNNN  NNNNNNNNNN
#    RRRRRRR        RRRRRRR      NNNNNNN   NNNNNNNNN      NNNNNNN   NNNNNNNNN
#   RRRRRRR         RRRRRRR     NNNNNNN    NNNNNNNN      NNNNNNN    NNNNNNNN 
#  RRRRRRR         RRRRRRR     NNNNNNN     NNNNNNN      NNNNNNN     NNNNNNN
#-----------------------------------------------------------------------------------------------------------------

In [15]:

#pd.DataFrame(gan).tail
#real_df_all.tail



In [16]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}
target_col_idx :  2
out_num_features :  1


In [17]:
#ctrl f -- 타겟

print(real_df_all.shape)
rnn_target_column = 'do'
print(rnn_target_column)



(26304, 918)
do


In [18]:

#train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.8, 0.1, 0.1)
#teg_check = 'train=8, test=1, val=1'

train_df, test_df, val_df = dataset_slice(real_df_all, 0.7, 0.1, 0.2)
teg_check = 'train=7, val=1, test=2'

print('train_df.shape : ', train_df.shape, 'val_df.shape : ', val_df.shape, 'test_df.shape:' ,test_df.shape)

train_df.shape :  (18412, 918) val_df.shape :  (5261, 918) test_df.shape: (2631, 918)


In [19]:
#fake_df.shape[0]/8760

In [20]:
idx = [2, 4, 5, 6, 7]
pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]
indices = {name: i for i, name in enumerate(idx)}
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)


label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}


In [21]:
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

In [22]:
WindowGenerator.make_dataset = make_dataset_water
multi_window = WindowGenerator(
    input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
    out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
    train_df=train_df, val_df=val_df, test_df=test_df)

model_path = "save/" + watershed + "models/" + file_parameters['watershed'] + '/'+  pa[indices[target_col_idx]]
print("save model path : ", model_path)

if __RNN_TRAINING__:
    if not os.path.exists(model_path):
        os.makedirs(model_path)

save model path :  save/yeong/models/yeong_pull/do/


In [23]:
#rnn_epochs, rnn_in_setps
__RNN_TRAINING__ = True
rnn_continue_train = False
rnn_epochs = 15
rnn_steps_per_epoch = 5

## 모델 학습

In [24]:
gru_model = model_gru(
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, 
    epochs=rnn_epochs, steps_per_epoch = rnn_steps_per_epoch,
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt", continue_train=rnn_continue_train)

Epoch 1/15
5/5 [==============================] - ETA: 0s - loss: 1.0231 - mean_absolute_error: 0.7993 - nse: -0.0962

KeyboardInterrupt: 

In [ ]:
rnn_predict_day = 2
rnn_predict_day

In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=real_df_all, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    predict_day = rnn_predict_day)
    #predict_day = 4)
    
print(model_path)
print('rnn_predict_day : ', rnn_predict_day + 1)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column , " ", out_features[0])
print("length : ",real_df_all.shape[0])
print(teg_check)

print('all_nse : ', val_nse['GRU'])
print('all_pbias : ', val_pbias['GRU'])
print('all_mae : ', _mae)
print('all_rmse : ', _rmse)
print('all_Rs : ', _Rs)
print('all_R : ', _R)


In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=train_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    predict_day = rnn_predict_day)
    #predict_day = 4)

print('----------------------------')
print('train_nse : ', val_nse['GRU'])
print('train_pbias : ', val_pbias['GRU'])
print('train_mae : ', _mae)
print('train_rmse : ', _rmse)
print('train_Rs : ', _Rs)
print('train_R : ', _R)

#print('nse_train : ', val_nse['GRU'])

In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=test_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    predict_day = rnn_predict_day)
    #predict_day = 4)

print('----------------------------')
print('val_nse : ', val_nse['GRU'])
print('val_pbias : ', val_pbias['GRU'])
print('val_mae : ', _mae)
print('val_rmse : ', _rmse)
print('val_Rs : ', _Rs)
print('val_R : ', _R)

In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    predict_day = rnn_predict_day)

print('----------------------------')
print('test_nse : ', val_nse['GRU'])
print('test_pbias : ', val_pbias['GRU'])
print('test_mae : ', _mae)
print('test_rmse : ', _rmse)
print('test_Rs : ', _Rs)
print('test_R : ', _R)

In [ ]:
#train_df.shape, val_df.shape, test_df.shape

In [ ]:
#-----------------------------------------------------------------------------------------------------------------
# 123SFACVTHYI78567UDBYUJM534523RDHGTHETYIE5I8I78O9P789J467UBDFGDASFSADCXVFDSWERTNERGRTTUTYAFGASDFSDBGFDF67E54563
# 64534FDSFVSDHT56SDFG656USRHSDYA3Y RTA WERGERGSDFGVTY34TASDF dfasdfghhgbcvbsdfgrwergfdsfvzxcvasdfaweqdqasdfderadf
# feasetttttttttttttttttttvxzcvxzvxcvxvnjfghjvb111nfghjdfhgdhthrtyyyyyyyyyyrtjliuioyuytrtyurtyughjuyityuityuityuijjty
# hgfgdrrtyetghfdghvcvbnhjfghjgggnfnmjhkuytlryewqwerasdfgghrthfgjyruhjyuihjknhybgtvfrcdezxcabnghrtyreeghghte45654y  
# tr345rgbbbdfghterty4534562345jhgy5462345234534n5345g345t3245d34f235345f345g34454f345t345er345234yt545y45tg45t534
#       SSSSSSSSSSSSSS         ASDFALKJIVAJLMSEFLKMALSKVJALKSRKLTGAKLSDGVLK   *****    ***************   
#            SSSSSSSSSSSSS               ******            ******              *****    *************       (*_*)  
#                   SSSSSSSS               ******            ******              *****    *****            ~( (>
#        SSSS         S  SSS               ******            ******              *****    *****            / (          
#      SSSS          S  SSSS               ******            ******              *****    *****                     
#     SSSSS            SSSS                ******             ******            ******    *****                     
#      SSSSSSSSSSSSSSSSSS                  ******              *********************      *****                   
#-----------------------------------------------------------------------------------------------------------------
stop = stop

In [ ]:

sub_flag = True
history_nse = []

bob = 0
b = 1

while sub_flag:
    gru_model = model_gru(
        #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
        window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=15, steps_per_epoch = 5,
        training_flag=True, checkpoint_path=model_path+"gru.ckpt", continue_train=False)
    
    #nse, _, _, _ = compa(
        #model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
        #predict_day = rnn_predict_day)
        #predict_day = 4)
    
    nse, _, _, _, _, _, _, _ = compa(
    model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
    
    if(nse > 0.5):
        sub_flag = False
   
    
    history_nse.append(nse)
    
    print('--------------------------')
    print(history_nse)
    print(nse)
    print('--------------------------')
    
        
    bob += b
    if(bob < -4):
        b = 1
    if(bob > 0):
        b = -1



In [ ]:

def compa2(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    width = input_width + label_width
    length = df.shape[0] - width
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    

    predictions = model(inputs).numpy()
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    

    
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    o_ = np.mean(label_day[:,predict_day,:])
    p_ = np.mean(pred_day[:,predict_day,:])
    
    oi = label_day[:,predict_day,:]
    pi = pred_day[:,predict_day,:]
    
    high = ((oi-o_)*(pi-p_)).sum()
    low = np.sqrt( ( (oi-o_)**2 ).sum() )
    low = low * np.sqrt( ( (pi-p_)**2 ).sum() )
    
    R = high/low
    RS = (R)**2
    
    return nse3, pbias3, inputs_day, pred_day, label_day, mae, rmse, RS, R

In [ ]:
__GAIN_TRAINING__, __RNN_TRAINING__

In [ ]:
#water_list = ['han', 'nak', 'geum', 'yeong']
#water_list = ['geum', 'yeong']
water_list = ['yeong']
for watererer in water_list:
    #file_parameters['watershed'] = 'geum'
    file_parameters['watershed'] = watererer
    parameters_path = parameters_dir+'/'+ file_parameters['watershed'] + '.json'
    with open(parameters_path, encoding='utf8') as json_file:
        parameters = json.load(json_file)

    data_parameters = parameters['data']
    interpolation_option = data_parameters['interpolation']
    colum_idx = data_parameters['columns']
    watershed = data_parameters['watershed']
    file_names = data_parameters['files']
    folder = data_parameters['directorys']
    for i in range(len(folder)):
        folder[i] = watershed+folder[i]

    if file_parameters['watershed'] == 'nak':
        run_num = range(len(folder))
    else:
        run_num = [0]

    real_df_all = pd.DataFrame([])
    target_all = target_mean = target_std = 0

    gain_val_performance = {}
    gain_performance = {}

    length = len(run_num)

    ddday = 31
    mmmonth = 12

    for i in range(length):

        idx = run_num[i]

        print('folder : ', data_path + folder[idx])

        df, times, mmmonth, ddday = make_dataframe(data_path+folder[idx], file_names[idx], 
                                   colum_idx[idx], interpolation=interpolation_option[idx],
                                  first_file_no=i, month=mmmonth, day=ddday)

        df_all, train_mean, train_std, df = normalize(df)


        if i == 0:
            dfff = df
            target_all = df_all
            target_std = train_std
            target_mean = train_mean
            start_year = str(times.iloc[0].year)
            end_year = str(times.iloc[-1].year)

        if interpolation_option[idx][0] == False:

            loadfiles = ['idx.npy', 'miss.npy', 'discriminator.h5', 'generator.h5']

            gain_calc_falg = True

            if __GAIN_TRAINING__ == True:
                gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
                #print(folder[idx], ': training ', 'Miss date save : ', gain_calc_falg)
            else:
                for file in loadfiles:
                    if os.path.isfile('save/' + folder[idx]+file):
                        shutil.copyfile('save/' + folder[idx]+file, 'save/'+file)
                    else:
                        if file == 'miss.npy':
                            gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')

            if gain_calc_falg == True:
                WindowGenerator.make_dataset = make_dataset_gain
                wide_window = WindowGenerator(input_width=gain_in_setps, label_width=gain_out_setps, shift=gain_shift,
                                              fill_no=gain_fill_no, miss_rate=gain_miss_rate, batch_size=gain_batch_size,
                                              train_df = df_all, val_df = df_all, test_df = df_all, df = df)

                gain = model_GAIN(shape=(gain_in_setps, df_all.shape[1]), gen_sigmoid=False, epochs=gain_epochs,
                                  training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')

                gain_val_performance[str(i)] = gain.evaluate(wide_window.val)
                gain_performance[str(i)] = gain.evaluate(wide_window.test, verbose=0)

                if __GAIN_TRAINING__ == True:
                    if not os.path.exists('save/' + folder[idx]):
                        os.makedirs('save/'+folder[idx])
                    for file in loadfiles:
                        shutil.copyfile('save/' + file, 'save/' + folder[idx] + file)

                        ori, gan = create_dataset_with_gain(gain=gain, shape=(gain_in_setps, df_all.shape[1]), df=df)

            else:
                gan = create_dataset_interpol(window=gain_in_setps, df=df)
        else:
            gan = create_dataset_interpol(window=gain_in_setps, df=df)

        if i == 0 :
            real_df_all = pd.DataFrame(gan)
        else:
            real_df_all = pd.concat([real_df_all, pd.DataFrame(gan)], axis=1)


    train_df, val_df, test_df = dataset_slice(real_df_all, 0.7, 0.1, 0.2)
    teg_check = 'train=7, val=1, test=2'

    label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
    print("label_columns_indices:")
    print(label_columns_indices)
    target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
    target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
    out_features = [target_col_idx]
    out_num_features = len(out_features)

    print("target_col_idx : ", target_col_idx)
    print('out_num_features : ', out_num_features)

    target_list = ['do', 'toc', 'tn', 'tp', 'chl-a']
    #target_list = ['tp', 'chl-a']
    for rnn_target_column in target_list:
        #print(real_df_all.shape)
        #rnn_target_column = 'chl-a'
        #print(rnn_target_column)

        idx = [2, 4, 5, 6, 7]
        pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]
        indices = {name: i for i, name in enumerate(idx)}
        target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
        label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
        print("label_columns_indices:")
        print(label_columns_indices)

        target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
        out_features = [target_col_idx]
        out_num_features = len(out_features)

        WindowGenerator.make_dataset = make_dataset_water
        multi_window = WindowGenerator(
            input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
            out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
            train_df=train_df, val_df=test_df, test_df=val_df)

        model_path = "save/" + watershed + "models/" + file_parameters['watershed'] + '/'+  pa[indices[target_col_idx]]
        #model_path = "save/" + watershed + "models/" + pa[indices[target_col_idx]]
        print("save model path : ", model_path)

        if __RNN_TRAINING__:
            if not os.path.exists(model_path):
                os.makedirs(model_path)

        gru_model = model_gru(
            window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, 
            epochs=rnn_epochs, steps_per_epoch = rnn_steps_per_epoch,
            training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt", continue_train=rnn_continue_train)

        temp_df = val_df
        data_type = 'val'
        for data_type in range(0,4,1):
            if data_type == 0:
                temp_df = real_df_all
                data_type = 'all'
            elif data_type == 1:
                temp_df = train_df
                data_type = 'train'
            elif data_type == 2:
                temp_df = val_df
                data_type = 'val'
            elif data_type == 3:
                temp_df = test_df
                data_type = 'test'

            for rnn_predict_day in range(2,5,1):
                _nse, _pbias, _input_day, _pred_day, _label_day, _mae, _rmse, _Rs, _R = compa2(
                    model=gru_model,df=temp_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
                    predict_day = rnn_predict_day)

                print(model_path)
                print("year : " + start_year + " ~ "+ end_year)
                print('run : ', run_num)
                print('target : ', rnn_target_column)
                print(teg_check)

                image_file_name = file_parameters['watershed']+'_'+rnn_target_column+'_'+data_type+'_'+str(rnn_predict_day+1)+'.png'
                print(image_file_name)

                outdata = {
                    'image_file_name' : image_file_name,
                    'watershed':file_parameters['watershed'],
                    'target':rnn_target_column,
                    'type':data_type,
                    'pred_day':rnn_predict_day+1,
                    'year':start_year + " ~ "+ end_year,
                    'nse':_nse, 
                    'pbias':_pbias,
                    'mae':_mae,
                    'rmse':_rmse,
                    'rs':_Rs,
                    'r':_R,
                }
                outdata = pd.DataFrame(outdata)
                outdata.to_csv('temp/data.csv', mode='a', header=False)

                plt.figure(figsize=(12,12),dpi=96)
                _input_index = np.array(range(_input_day.shape[0]))
                _label_index = _input_index + (7 + rnn_predict_day)
                plt.plot(_input_index, _input_day[:, 0, :], label='input')
                plt.plot(_label_index, _label_day[:, rnn_predict_day, :], label='label')
                plt.plot(_label_index, _pred_day[:, rnn_predict_day, :], label='pred')
                plt.legend()
                plt.savefig('temp/'+image_file_name)

    print('end')

In [25]:

target_list = ['do', 'toc', 'tn', 'tp', 'chl-a']
for rnn_target_column in target_list:
    #print(real_df_all.shape)
    #rnn_target_column = 'do'
    #print(rnn_target_column)

    idx = [2, 4, 5, 6, 7]
    pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]
    indices = {name: i for i, name in enumerate(idx)}
    target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
    label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
    print("label_columns_indices:")
    print(label_columns_indices)

    target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
    out_features = [target_col_idx]
    out_num_features = len(out_features)

    WindowGenerator.make_dataset = make_dataset_water
    multi_window = WindowGenerator(
        input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
        out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
        train_df=train_df, val_df=val_df, test_df=test_df)

    model_path = "save/" + watershed + "models/" + file_parameters['watershed'] + '/'+  pa[indices[target_col_idx]]
    print("save model path : ", model_path)

    if __RNN_TRAINING__:
        if not os.path.exists(model_path):
            os.makedirs(model_path)

    #rnn_epochs, rnn_in_setps
    __RNN_TRAINING__ = True
    rnn_continue_train = False
    rnn_epochs = 15
    rnn_steps_per_epoch = 5

    gru_model = model_gru(
        window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, 
        epochs=rnn_epochs, steps_per_epoch = rnn_steps_per_epoch,
        training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt", continue_train=rnn_continue_train)

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}
save model path :  save/yeong/models/yeong_pull/do/
Epoch 1/15
5/5 [==============================] - 111s 22s/step - loss: 1.0740 - mean_absolute_error: 0.8269 - nse: -0.1869 - val_loss: 0.9991 - val_mean_absolute_error: 0.8167 - val_nse: -0.1134
Epoch 2/15
5/5 [==============================] - 101s 20s/step - loss: 0.7722 - mean_absolute_error: 0.6889 - nse: 0.0881 - val_loss: 0.9037 - val_mean_absolute_error: 0.7775 - val_nse: -0.0075
Epoch 3/15
5/5 [==============================] - 102s 20s/step - loss: 0.6029 - mean_absolute_error: 0.6080 - nse: 0.3085 - val_loss: 0.7767 - val_mean_absolute_error: 0.7122 - val_nse: 0.1336
Epoch 4/15
5/5 [==============================] - 101s 20s/step - loss: 0.5162 - mean_absolute_error: 0.5553 - nse: 0.3695 - val_loss: 0.7172 - val_mean_absolute_e

Epoch 4/15
5/5 [==============================] - 101s 20s/step - loss: 0.2473 - mean_absolute_error: 0.3784 - nse: 0.4917 - val_loss: 0.2105 - val_mean_absolute_error: 0.3679 - val_nse: 0.4650
Epoch 5/15
5/5 [==============================] - 101s 20s/step - loss: 0.2099 - mean_absolute_error: 0.3461 - nse: 0.5764 - val_loss: 0.2065 - val_mean_absolute_error: 0.3619 - val_nse: 0.4750
Epoch 6/15
5/5 [==============================] - 101s 20s/step - loss: 0.1725 - mean_absolute_error: 0.3147 - nse: 0.6300 - val_loss: 0.2125 - val_mean_absolute_error: 0.3756 - val_nse: 0.4594
Epoch 7/15
5/5 [==============================] - 102s 20s/step - loss: 0.1695 - mean_absolute_error: 0.3095 - nse: 0.6809 - val_loss: 0.2018 - val_mean_absolute_error: 0.3516 - val_nse: 0.4839
Epoch 8/15
5/5 [==============================] - 102s 20s/step - loss: 0.1470 - mean_absolute_error: 0.2932 - nse: 0.6719 - val_loss: 0.2087 - val_mean_absolute_error: 0.3568 - val_nse: 0.4665
Epoch 9/15
5/5 [==============

Epoch 10/15
5/5 [==============================] - 102s 20s/step - loss: 0.4356 - mean_absolute_error: 0.4531 - nse: 0.7726 - val_loss: 1.7035 - val_mean_absolute_error: 0.9124 - val_nse: -0.1180
Epoch 11/15
5/5 [==============================] - 102s 20s/step - loss: 0.5302 - mean_absolute_error: 0.4916 - nse: 0.7240 - val_loss: 1.5849 - val_mean_absolute_error: 0.8708 - val_nse: -0.0391
Epoch 12/15
5/5 [==============================] - 101s 20s/step - loss: 0.5208 - mean_absolute_error: 0.4920 - nse: 0.7482 - val_loss: 1.9743 - val_mean_absolute_error: 0.9656 - val_nse: -0.3001
Epoch 13/15
5/5 [==============================] - 102s 20s/step - loss: 0.4912 - mean_absolute_error: 0.4833 - nse: 0.8090 - val_loss: 1.7490 - val_mean_absolute_error: 0.9145 - val_nse: -0.1489
Epoch 14/15
5/5 [==============================] - 101s 20s/step - loss: 0.3880 - mean_absolute_error: 0.4221 - nse: 0.7667 - val_loss: 1.6484 - val_mean_absolute_error: 0.8869 - val_nse: -0.0806
Epoch 15/15
5/5 [===

In [ ]:
#water_list = ['han', 'nak', 'geum', 'yeong']
#water_list = ['geum', 'yeong']

for ddddd in [1]:
    label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
    print("label_columns_indices:")
    print(label_columns_indices)
    target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
    target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
    out_features = [target_col_idx]
    out_num_features = len(out_features)

    print("target_col_idx : ", target_col_idx)
    print('out_num_features : ', out_num_features)

    target_list = ['do', 'toc', 'tn', 'tp', 'chl-a']
    #target_list = ['tp', 'chl-a']
    for rnn_target_column in target_list:
        #print(real_df_all.shape)
        #rnn_target_column = 'chl-a'
        #print(rnn_target_column)

        idx = [2, 4, 5, 6, 7]
        pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]
        indices = {name: i for i, name in enumerate(idx)}
        target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
        label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
        print("label_columns_indices:")
        print(label_columns_indices)

        target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
        out_features = [target_col_idx]
        out_num_features = len(out_features)

        WindowGenerator.make_dataset = make_dataset_water
        multi_window = WindowGenerator(
            input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
            out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
            train_df=train_df, val_df=test_df, test_df=val_df)

        model_path = "save/" + watershed + "models/" + file_parameters['watershed'] + '/'+  pa[indices[target_col_idx]]
        #model_path = "save/" + watershed + "models/" + pa[indices[target_col_idx]]
        print("save model path : ", model_path)

        if __RNN_TRAINING__:
            if not os.path.exists(model_path):
                os.makedirs(model_path)

        gru_model = model_gru(
            window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, 
            epochs=rnn_epochs, steps_per_epoch = rnn_steps_per_epoch,
            training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt", continue_train=rnn_continue_train)

        temp_df = val_df
        data_type = 'val'
        for data_type in range(0,4,1):
            if data_type == 0:
                temp_df = real_df_all
                data_type = 'all'
            elif data_type == 1:
                temp_df = train_df
                data_type = 'train'
            elif data_type == 2:
                temp_df = val_df
                data_type = 'val'
            elif data_type == 3:
                temp_df = test_df
                data_type = 'test'

            for rnn_predict_day in range(2,5,1):
                _nse, _pbias, _input_day, _pred_day, _label_day, _mae, _rmse, _Rs, _R = compa2(
                    model=gru_model,df=temp_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
                    predict_day = rnn_predict_day)

                print(model_path)
                print("year : " + start_year + " ~ "+ end_year)
                print('run : ', run_num)
                print('target : ', rnn_target_column)
                print(teg_check)

                image_file_name = file_parameters['watershed']+'_'+rnn_target_column+'_'+data_type+'_'+str(rnn_predict_day+1)+'.png'
                print(image_file_name)

                outdata = {
                    'image_file_name' : image_file_name,
                    'watershed':file_parameters['watershed'],
                    'target':rnn_target_column,
                    'type':data_type,
                    'pred_day':rnn_predict_day+1,
                    'year':start_year + " ~ "+ end_year,
                    'nse':_nse, 
                    'pbias':_pbias,
                    'mae':_mae,
                    'rmse':_rmse,
                    'rs':_Rs,
                    'r':_R,
                }
                outdata = pd.DataFrame(outdata)
                outdata.to_csv('temp/data.csv', mode='a', header=False)

                plt.figure(figsize=(12,12),dpi=96)
                _input_index = np.array(range(_input_day.shape[0]))
                _label_index = _input_index + (7 + rnn_predict_day)
                plt.plot(_input_index, _input_day[:, 0, :], label='input')
                plt.plot(_label_index, _label_day[:, rnn_predict_day, :], label='label')
                plt.plot(_label_index, _pred_day[:, rnn_predict_day, :], label='pred')
                plt.legend()
                plt.savefig('temp/'+image_file_name)

    print('end')